In [50]:
from skimage.segmentation import slic
from skimage import feature
import skimage.io as io
from skimage import transform , data
import skimage.transform
from sklearn.metrics import precision_score, recall_score,f1_score, confusion_matrix ,accuracy_score


In [51]:
# import the necessary packages
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2,os
from skimage.io import imread 
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB


### Methods to Use Amoung File

In [53]:
def getScores(model_,X_train_, y_train_, X_val_, y_val_, y_pred_):
    """
    This method gets all the needed scores (accuracy, precision, recall and f1) 
    from the training/validation sets after getting the model, x train and
    validation, y train and validation, and the y predicted
    """
    print("Training Score ",model_.score(X_train_, y_train_))
    print("Validation Score ",model_.score(X_val_, y_val_))
    print("Precision: {:6.4f},   Recall: {:6.4f}, f1: {:6.4f}".format(precision_score(y_val_, y_pred_, average='micro'), 
                                                         recall_score(y_val_, y_pred_, average='micro'),f1_score(y_val_,y_pred_,average='micro')))

In [30]:


def make_confusion_matrix(model, threshold=0.5):
    """
    This method draws the confusion matrixs for each given model
    """
    y_predict = (model.predict_proba(new_X_val)[:, 2] >= threshold)
    avtivity_confusion = confusion_matrix(y_val.astype(str), y_predict.astype(str))
    plt.figure(dpi=100)
    sns.heatmap(avtivity_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels= new_X_val,
           yticklabels= new_X_val);
    plt.xlabel('prediction')
    plt.ylabel('actual')

In [54]:
batch_size = 32
imageSize = 32
target_dims = (imageSize, imageSize, 3)
num_classes = 3
train_len = 8982
train_dir = r'C:\Users\Computer\Desktop\flask_app\Dataset'
def get_data(folder):
    X = np.empty((train_len, imageSize, imageSize, 3), dtype=np.float32)
    y = np.empty((train_len,), dtype=np.int)
    cnt = 0
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if folderName in ['with_mask']:
                label = 0
            elif folderName in ['without_mask']:
                label = 1
            elif folderName in ['mask_weared_incorrect']:
                label = 2

            for image_filename in os.listdir(folder + '\\' + folderName):
                img_file = cv2.imread(folder + '\\' + folderName + '\\' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file).reshape((-1, imageSize, imageSize, 3))
                    
                    X[cnt] = img_arr
                    y[cnt] = label
                    cnt += 1
    return X,y
X, y = get_data(train_dir)

In [55]:
X.shape , y.shape 

((8982, 32, 32, 3), (8982,))

In [56]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=.2, random_state=77,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=77)

In [57]:
print(f"\nTraining data: {X_train.shape},  labels: {y_train.shape}")
print(f"Validation data: {X_val.shape},  labels: {y_val.shape}")
print(f"Testing data: {X_test.shape},  labels: {y_test.shape}")


Training data: (7185, 32, 32, 3),  labels: (7185,)
Validation data: (898, 32, 32, 3),  labels: (898,)
Testing data: (899, 32, 32, 3),  labels: (899,)


In [58]:
new_X_train = X_train.reshape(7185, 32 * 32 * 3)
new_X_val = X_val.reshape(898, 32 * 32 * 3)
#new_X_test = X_test.reshape(898, 32 * 32 * 3)

### Baseline 
### 1-  Logistic Regression 

#### Model Fitting

In [13]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear')
model = model.fit(new_X_train, y_train)

#### Model Prediction

In [14]:
y_pred = model.predict(new_X_val)

#### Model Scores

In [15]:
getScores(model, new_X_train, y_train, new_X_val, y_val, y_pred)

Training Score  0.9611691022964509
Validation Score  0.9220489977728286
Precision: 0.9220,   Recall: 0.9220, f1: 0.9220


###  2- Random Forest Classifier

#### Model Fitting

In [18]:
model = RandomForestClassifier(n_estimators=40)
model = model.fit(new_X_train, y_train)

#### Model Prediction

In [19]:
y_pred = model.predict(new_X_val)

#### Model Scores

In [20]:
getScores(model, new_X_train, y_train, new_X_val, y_val, y_pred)

Training Score  1.0
Validation Score  0.9398663697104677
Precision: 0.9399,   Recall: 0.9399, f1: 0.9399


### 3- GaussianNB

#### Model Fitting

In [21]:
model = GaussianNB().fit(new_X_train, y_train)

#### Model Prediction

In [22]:
y_pred = model.predict(new_X_val)

#### Model Scores

In [23]:
getScores(model, new_X_train, y_train, new_X_val, y_val, y_pred)

Training Score  0.7421016005567154
Validation Score  0.7427616926503341
Precision: 0.7428,   Recall: 0.7428, f1: 0.7428


### 4- BernoulliNB

#### Model Fitting

In [61]:
model = BernoulliNB().fit(new_X_train, y_train)

#### Model Prediction

In [62]:
y_pred = model.predict(new_X_val)

#### Model Scores

In [63]:
getScores(model, new_X_train, y_train, new_X_val, y_val, y_pred)

Training Score  0.3684064022268615
Validation Score  0.34521158129175944
Precision: 0.3452,   Recall: 0.3452, f1: 0.3452
